In [1]:
prog = "1+1"

"1+1"

In [2]:
ex1 = Meta.parse(prog)

:(1 + 1)

In [3]:
typeof(ex1)

Expr

In [4]:
ex1.head

:call

In [5]:
ex1.args

3-element Vector{Any}:
  :+
 1
 1

In [6]:
ex2 = Expr(:call, :+, 1, 1)

:(1 + 1)

In [7]:
ex1 == ex2

true

In [9]:
dump(ex2)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1


In [10]:
ex3 = Meta.parse("(4+4)/2")

:((4 + 4) / 2)

In [12]:
Meta.show_sexpr(ex3)

(:call, :/, (:call, :+, 4, 4), 2)

In [13]:
s = :foo

:foo

In [14]:
typeof(s)

Symbol

In [16]:
:foo == Symbol("foo")

true

In [17]:
Symbol(:var,'_',:sym)

:var_sym

In [18]:
:(:)

:(:)

In [7]:
ex = :(::)
typeof(ex)

Symbol

Expressions and evalaution

In [20]:
ex = :(a+b*c+1)

:(a + b * c + 1)

In [21]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((4,))
    1: Symbol +
    2: Symbol a
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Symbol b
        3: Symbol c
    4: Int64 1


In [22]:
a = 1
ex = :($a + b*c +1)

:(1 + b * c + 1)

In [23]:
ex = quote 
    x = 1
    y = 2
    x+y
end

quote
    #= In[23]:2 =#
    x = 1
    #= In[23]:3 =#
    y = 2
    #= In[23]:4 =#
    x + y
end

In [24]:
a = 1
ex= :($a + b)

:(1 + b)

In [25]:
$a + b

LoadError: syntax: "$" expression outside quote around In[25]:1

In [27]:
args = [:x, :y, :z]
:(f(1, $(args...)))


:(f(1, x, y, z))

In [28]:
x = :(1+2)


:(1 + 2)

In [29]:
e = quote quote $x end end

quote
    #= In[29]:1 =#
    $(Expr(:quote, quote
    #= In[29]:1 =#
    $(Expr(:$, :x))
end))
end

In [32]:
eval(eval(e))

3

In [34]:
dump(Meta.parse(":(1+2)"))



Expr
  head: Symbol quote
  args: Array{Any}((1,))
    1: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Int64 1
        3: Int64 2


In [36]:
a =  :(:(1+2))
dump(a)

Expr
  head: Symbol quote
  args: Array{Any}((1,))
    1: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Int64 1
        3: Int64 2


In [37]:
ex1 = :(1+2)

:(1 + 2)

In [38]:
eval(ex1)

3

In [39]:
ex = :(a+b)

:(a + b)

In [40]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol a
    3: Symbol b


In [41]:
eval(ex)

LoadError: UndefVarError: b not defined

In [42]:
a = 1; b = 2;
eval(ex)

3

In [43]:
ex = :(x=1)


:(x = 1)

In [44]:
x


:(1 + 2)

In [47]:
eval(ex)

1

In [48]:
x


1

In [49]:
a = 1;
ex =  Expr(:call, :+, a, :b)

:(1 + b)

In [50]:
a = 0; b= 2


2

In [51]:
eval(x)

1

In [52]:
function math_expr(op, op1, op2)
    expr = Expr(:call, op, op1, op2)
    return expr
end

math_expr (generic function with 1 method)

In [53]:
ex = math_expr(:+, 1, Expr(:call, :*, 4, 5))

:(1 + 4 * 5)

In [55]:
eval(ex)

21

maCROS

In [62]:
macro sayhello()
    return :(println("Hello world"))
end

@sayhello (macro with 1 method)

In [63]:
@sayhello()


Hello world


In [64]:
macro sayhello(name)
    return :(println("Hello, ", $name))
end

@sayhello (macro with 2 methods)

In [66]:
@sayhello("Aditya")

Hello, Aditya


In [68]:
wx =  macroexpand(Main, :(@sayhello("human")))

:(Main.println("Hello, ", "human"))

In [69]:
@macroexpand @sayhello "human"

:(Main.println("Hello, ", "human"))

In [70]:
macro twosetp(arg)
    println("I execute at parse time. The argument is: ", arg)
    return :(println("I execute at runtime. The argument is:") $arg)
end

@twosetp (macro with 1 method)

In [72]:
ex = macroexpand(Main, :(@twosetp :(1,2,3)))

I execute at parse time. The argument is: :((1, 2, 3))


:(Main.println("I execute at runtime. The argument is:") $ Main.arg)

In [2]:
macro showarg(x)
    show(x)
end


@showarg (macro with 1 method)

In [3]:
@showarg(a)

:a

In [4]:
@showarg(1+1)

:(1 + 1)

In [5]:
@showarg(println("Yo!"))

:(println("Yo!"))

In [6]:
:x


:x

In [7]:
string(:x)

"x"

In [8]:
$x


LoadError: syntax: "$" expression outside quote around In[8]:1

In [10]:
:($(string(:x)))

"x"

In [11]:
:(string(:x))

:(string(:x))

In [12]:
:("x")

"x"

In [27]:
macro assert(ex)
    return :($ex ? nothing : throw(AssertionError($(string(ex)))))
end

@assert (macro with 1 method)

In [21]:
@assert 1 == 1.0

In [28]:
@assert 1 == 0

LoadError: AssertionError: 1 == 0

In [29]:
string(:(1 == 1.0))

"1 == 1.0"

In [30]:
?AssertionError


search: AssertionError



```
AssertionError([msg])
```

The asserted condition did not evaluate to `true`. Optional argument `msg` is a descriptive error string.

# Examples

```jldoctest
julia> @assert false "this is not true"
ERROR: AssertionError: this is not true
```

`AssertionError` is usually thrown from [`@assert`](@ref).


In [34]:
throw(AssertionError("1=1.0"))

LoadError: AssertionError: 1=1.0

In [27]:
ex = :(1 == 10.0)
a = string(ex)
ex2 = :( $ex ? nothing : throw(AssertionError($(a))) )
dump(ex2)

Expr
  head: Symbol if
  args: Array{Any}((3,))
    1: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol ==
        2: Int64 1
        3: Float64 10.0
    2: Symbol nothing
    3: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol throw
        2: Expr
          head: Symbol call
          args: Array{Any}((2,))
            1: Symbol AssertionError
            2: String "1 == 10.0"


In [14]:
ex = :(1==10)
eval(:(string(ex)))

"1 == 10"

Expr
  head: Symbol if
  args: Array{Any}((3,))
    1: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol ==
        2: Int64 1
        3: Float64 10.0
    2: Symbol nothing
    3: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol throw
        2: Expr
          head: Symbol call
          args: Array{Any}((2,))
            1: Symbol AssertionError
            2: String "1 == 10.0"


In [31]:
macro assert(ex, msgs...)
    msg_body = isempty(msgs) ? ex : msgs[1]
    msg = string(msg_body)
    print("Parse time:", msg )
    return :($ex ? nothing : throw(AssertionError($msg)))
end

@assert (macro with 1 method)

In [32]:
@macroexpand @assert a == b

Parse time:a == b

:(if Main.a == Main.b
      Main.nothing
  else
      Main.throw(Main.AssertionError("a == b"))
  end)

In [33]:
@macroexpand @assert a==b "a should equal b!"

Parse time:a should equal b!

:(if Main.a == Main.b
      Main.nothing
  else
      Main.throw(Main.AssertionError("a should equal b!"))
  end)